In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg() 

In [ ]:
sys.path.append('/home/cesga/gferro/NEASQC/PhaseAmplification/')

In [ ]:
from QuantumMultiplexors_Module_examples import expectation_loading_data
from PhaseAmplification_Module import load_q_gate

In [ ]:
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
from qat.lang.AQASM import H, S, T, PH
def IQFT_old(q_prog, q_gate, q_aux, c_bits, l):
    """
    Implements a iterative step of the Iterative Phase Estimation (IPE) algorithm.
    
    Parameters
    ----------
    
    q_prog : QLM program
        QLM Program where the unitary operator will be applied
    q_gate : QLM AbstractGate
        QLM implementation of the unitary operator. We want estimate the autovalue \theta of this operator
    q_aux : QLM qbit
        auxiliar qbit for IPE. This qbit will be the control for application of the unitary operator to the principal
        qbits of the program. Aditionally will be the target qbit for the classical bit controlled rotation. This qbit
        will be reset at the end of the step. 
    c_bits : list
        list with the classical bits allocated for phase estimation
    l : int
        iteration step of the IPE algorithm
    
    """
    
    #Getting the principal qbits
    q_bits = q_prog.registers[0]
    
    #First apply a Haddamard Gate to auxiliar qbit
    q_prog.apply(H, q_aux)
    #number of bits for codify phase
    m = len(c_bits)

    #Number of controlled application of the unitary operator by auxiliar qbit over the principal qbits
    unitary_applications = int(2**(m-l-1))
    print('unitary_applications: {}'.format(unitary_applications))
    for i in range(unitary_applications):
        q_prog.apply(q_gate.ctrl(), q_aux, q_bits)
        
    print('m: {}. l: {}'.format(m,l))
    
    for j in range(m-l+1, m+1, 1):
        theta = 2**(m-l-j+1)
        print('j: {}. theta: {}'.format(j-1, theta))
        q_prog.cc_apply(c_bits[j-1], PH(-(np.pi/2.0)*theta), q_aux)        
        
    q_prog.apply(H, q_aux)
    print(m-l-1)
    q_prog.measure(q_aux, c_bits[m-l-1])
    q_prog.reset(q_aux)        

    
from qat.comm.datamodel.ttypes import OpType

In [ ]:
from qat.lang.AQASM import H, S, T, PH
def IQFT(q_prog, q_gate, q_aux, c_bits, l):
    """
    Implements a iterative step of the Iterative Phase Estimation (IPE) algorithm.
    
    Parameters
    ----------
    
    q_prog : QLM program
        QLM Program where the unitary operator will be applied
    q_gate : QLM AbstractGate
        QLM implementation of the unitary operator. We want estimate the autovalue \theta of this operator
    q_aux : QLM qbit
        auxiliar qbit for IPE. This qbit will be the control for application of the unitary operator to the principal
        qbits of the program. Aditionally will be the target qbit for the classical bit controlled rotation. This qbit
        will be reset at the end of the step. 
    c_bits : list
        list with the classical bits allocated for phase estimation
    l : int
        iteration step of the IPE algorithm
    
    """
    
    q_prog.reset(q_aux)
    #Getting the principal qbits
    q_bits = q_prog.registers[0]
    
    #First apply a Haddamard Gate to auxiliar qbit
    q_prog.apply(H, q_aux)
    #number of bits for codify phase
    m = len(c_bits)

    #Number of controlled application of the unitary operator by auxiliar qbit over the principal qbits
    unitary_applications = int(2**(m-l-1))
    print('unitary_applications: {}'.format(unitary_applications))
    for i in range(unitary_applications):
        q_prog.apply(q_gate.ctrl(), q_aux, q_bits)
        
    print('m: {}. l: {}'.format(m,l))
    
    for j in range(m-l+1, m+1, 1):
        theta = 2**(m-l-j+1)
        print('j: {}. theta: {}'.format(j-1, theta))
        if c_bits[j-1] == True:
            q_prog.apply(PH((np.pi/2.0)*theta), q_aux)
        #q_prog.cc_apply(c_bits[j-1], PH(-(np.pi/2.0)*theta), q_aux)        
        
    q_prog.apply(H, q_aux)
    #print(m-l-1)
    #q_prog.measure(q_aux, c_bits[m-l-1])
    
         

    
from qat.comm.datamodel.ttypes import OpType

In [ ]:
#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

In [ ]:
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
Q_Gate = load_q_gate(P_Gate, R_gate)

In [ ]:
circuit = Qprog.to_circ(submatrices_only=True) 
%qatdisplay circuit

In [ ]:
NumberOfClassicalBits = 6
qAux = Qprog.qalloc(1)
#cbits = Qprog.calloc(NumberOfClassicalBits)
cbits = [False for i in range(NumberOfClassicalBits)]

In [ ]:
ListOfThetas = []
for rep in range(10):
    Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
    Q_Gate = load_q_gate(P_Gate, R_gate)
    NumberOfClassicalBits = 6
    qAux = Qprog.qalloc(1)
    #cbits = Qprog.calloc(NumberOfClassicalBits)
    cbits = [False for i in range(NumberOfClassicalBits)]

    ListOfPdfs = []
    for l in range(len(cbits)):
        IQFT(Qprog, Q_Gate, qAux, cbits, l)
        circuit = Qprog.to_circ(submatrices_only=True)
        #%qatdisplay circuit
        #meas_gates = [i for i, o in enumerate(circuit.ops) if o.type==OpType.MEASURE]
        job = circuit.to_job(qubits=qAux, nbshots=1)
        result = run_job(lineal_qpu.submit(job))
        pdf_r = postprocess_results(result)
        cbit_measured = bool(pdf_r.iloc[pdf_r['Probability'].idxmax()]['States'].int)
        cbits[NumberOfClassicalBits-l-1] = cbit_measured

    cbits.reverse()
    listToStr = ''.join([str(int(elem)) for elem in cbits])
    print('listToStr: {}'.format(listToStr))
    print('M= {}'.format(int(listToStr,2)))
    theta  = int(listToStr,2)*np.pi/(2**len(cbits))
    print('theta: {}'.format(theta))

    ListOfThetas.append(theta)

# OTRA

In [ ]:
#number of Qbits for the circuit
n_qbits = 8
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

In [ ]:
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
Q_Gate = load_q_gate(P_Gate, R_gate)

In [ ]:
from qat.lang.AQASM import H, S, T, PH
from qat.comm.datamodel.ttypes import OpType
def IQFT_step(q_prog, q_gate, q_aux, c_bits, l):
    """
    Implements a iterative step of the Iterative Phase Estimation (IPE) algorithm.
    
    Parameters
    ----------
    
    q_prog : QLM program
        QLM Program where the unitary operator will be applied
    q_gate : QLM AbstractGate
        QLM implementation of the unitary operator. We want estimate the autovalue \theta of this operator
    q_aux : QLM qbit
        auxiliar qbit for IPE. This qbit will be the control for application of the unitary operator to the principal
        qbits of the program. Aditionally will be the target qbit for the classical bit controlled rotation. This qbit
        will be reset at the end of the step. 
    c_bits : list
        list with the classical bits allocated for phase estimation
    l : int
        iteration step of the IPE algorithm
    
    """
    
    q_prog.reset(q_aux)
    #Getting the principal qbits
    q_bits = q_prog.registers[0]
    
    #First apply a Haddamard Gate to auxiliar qbit
    q_prog.apply(H, q_aux)
    #number of bits for codify phase
    m = len(c_bits)

    #Number of controlled application of the unitary operator by auxiliar qbit over the principal qbits
    unitary_applications = int(2**(m-l-1))
    print('unitary_applications: {}'.format(unitary_applications))
    for i in range(unitary_applications):
        q_prog.apply(q_gate.ctrl(), q_aux, q_bits)
        
    print('m: {}. l: {}'.format(m,l))
    
    for j in range(m-l+1, m+1, 1):
        theta = 2**(m-l-j+1)
        print('j: {}. theta: {}'.format(j-1, theta))
        if c_bits[j-1] == True:
            q_prog.apply(PH((np.pi/2.0)*theta), q_aux)
        #q_prog.cc_apply(c_bits[j-1], PH(-(np.pi/2.0)*theta), q_aux)        
        
    q_prog.apply(H, q_aux)
    #print(m-l-1)
    #q_prog.measure(q_aux, c_bits[m-l-1])
    
    #Create Circuit
    circuit = q_prog.to_circ(submatrices_only=True)
    #create job
    job = circuit.to_job(qubits=qAux, nbshots=1)
    #Submit job
    result = run_job(lineal_qpu.submit(job))
    #postproccess result
    pdf_r = postprocess_results(result)
    #Get classical bit measurement
    cbit_measured = bool(pdf_r.iloc[pdf_r['Probability'].idxmax()]['States'].int)
    #stored measured bit
    c_bits[m-l-1] = cbit_measured
    
         

    


In [ ]:
circuit = Qprog.to_circ(submatrices_only=True) 
%qatdisplay circuit

In [ ]:
NumberOfClassicalBits = 6
qAux = Qprog.qalloc(1)
#cbits = Qprog.calloc(NumberOfClassicalBits)
cbits = [False for i in range(NumberOfClassicalBits)]
for l in range(len(cbits)):
    IQFT_step(Qprog, Q_Gate, qAux, cbits, l)
cbits.reverse()
listToStr = ''.join([str(int(elem)) for elem in cbits])
print('listToStr: {}'.format(listToStr))
print('M= {}'.format(int(listToStr,2)))
theta  = int(listToStr,2)*np.pi/(2**len(cbits))
print('theta: {}'.format(theta))    

In [ ]:
cbits

In [ ]:
ListOfThetas

In [ ]:
[np.sin(theta)**2 for theta in ListOfThetas]

In [ ]:
cbits.reverse()

In [ ]:
listToStr = ''.join([str(int(elem)) for elem in cbits])
print('listToStr: {}'.format(listToStr))
print('M= {}'.format(int(listToStr,2)))
theta  = int(listToStr,2)*np.pi/(2**len(cbits))
print('theta: {}'.format(theta))
print(np.sin(theta)**2)

In [ ]:
sum(f_X*p_X)

In [ ]:
ListOfPdfs.reverse()

In [ ]:
ListOfPdfs

In [ ]:
cbits.reverse()

In [ ]:
cbits

In [ ]:
listToStr = ''.join([str(int(elem)) for elem in cbits])

In [ ]:
listToStr

In [ ]:
int(listToStr,2)

In [ ]:
theta  = int(listToStr,2)*np.pi/(2**len(cbits))

In [ ]:
theta

In [ ]:
np.sin(theta)**2

In [ ]:
sum(f_X*p_X)

In [ ]:
np.arcsin(sum(f_X*p_X)**0.5)

In [ ]:
[i for i in cbits]

In [ ]:
int_list = list(range(2**len(self.cbit_solution)))

In [ ]:
bits_list = [format(i, "b").zfill(len(cbits)) for i in range(2**len(cbits))]

In [ ]:
bits_list

In [ ]:
for l in range(len(cbits)):
    print('l: {}'.format(l))
    IQFT(Qprog, Q_Gate, qAux, cbits, l)
    circuit = Qprog.to_circ(submatrices_only=True)
    meas_gates = [i for i, o in enumerate(circuit.ops) if o.type==OpType.MEASURE]
    job = circuit.to_job(qubits=qAux)
    result_ = result[0]
    cbit = [r.cbits[0] for r in result_.intermediate_measurements if r.gate_pos == meas_gates[0]][0]
    print(cbit)

In [ ]:
meas_gates

In [ ]:
circuit = Qprog.to_circ(submatrices_only=True) 
%qatdisplay circuit --file kk.svg --svg

In [ ]:
job = circuit.to_job(qubits=qAux)
result = lineal_qpu.submit(job).join()


meas_gates = [i for i, o in enumerate(circuit.ops) if o.type==OpType.MEASURE]

In [ ]:
if len(result) ==1 :
    r = result[0]
    step_im = r.intermediate_measurements
    cbits_ = [[im.cbits[0], im.probability] for im in step_im if im.gate_pos in meas_gates]
    cbits_.reverse()

In [ ]:
def get_probability(bit, clasical_bits):
    """
    Calculates the probability of a string of bits bassed 
    on probabilities for each individual bit
    
    Parameters
    ----------
    
    bit : str
        strign of bits that represent an integer number
    clasical_bits : list
        it contains for each position a bolean value and the probability for it
        len(clasical_bits) == len(bit)
        classica_bits[i][0] : bolean value
        classica_bits[i][1] : probability of correspondient bolean value
    
    Returns
    ----------
    
    total_probability : float
        Probability of getting input bit having the 
        probability configuration of clasical_bits
        
    """
    p_ = []
    for i, b_ in enumerate(bit):
        #print(i, b_)
        
        if clasical_bits[i][0] == bool(int(b_)):
            #print('cierto')
            p_.append(clasical_bits[i][1])
        else:
            #print('false')
            p_.append(1.0-clasical_bits[i][1])
        #print(p)
    total_probability = np.prod(p_)
    return total_probability
        

In [ ]:
#posible integers that can be represented by len(cbits_)
int_list = [i for i in range(2**len(cbits_))]
#string bolean representation of each posible integer
bits_list = [format(i, "b").zfill(len(cbits_)) for i in range(2**len(cbits_))]
#probabilit of each integer based on probabilities of cbits_
prob_list = []
for b in bits_list:
    prob_list.append(get_probability(b, cbits_))

meas_cbits = pd.DataFrame({
    'Int' : int_list,
    'Bol' : bits_list,
    'Probability': prob_list
})
#posible thetas based the posible integers
meas_cbits['theta'] = meas_cbits['Int']*np.pi/(2**len(cbits_))
#Desired Integral
meas_cbits['Integral'] = np.sin(meas_cbits['theta'])**2

In [ ]:
%matplotlib inline
plt.plot(meas_cbits['theta'], meas_cbits['Probability'])
plt.xlabel('theta')
plt.ylabel('Probability')

In [ ]:
meas_cbits.sort_values('Probability', ascending=False).iloc[:10]

In [ ]:
meas_cbits.sort_values('Probability', ascending=False).iloc[:10]

In [ ]:
np.arcsin(sum(p_X*f_X)**0.5)

In [ ]:
np.pi-2.184389

In [ ]:
np.pi

## Comparo Qiskit

In [ ]:
from qat.lang.AQASM import Program, H, X, PH
nq = 2
nc = 3
q_prog = Program()
q_bits = q_prog.qalloc(nq)
cbits = q_prog.calloc(nc)
for i in range(nq):
    q_prog.apply(X, q_bits[i])
qAux = q_prog.qalloc(1)
q_gate = PH(np.pi/4.0).ctrl()
for l in range(len(cbits)):
    print('l: {}'.format(l))
    IQFT(q_prog, q_gate, qAux, cbits, l)
circuit = q_prog.to_circ(submatrices_only=True) 
%qatdisplay circuit    